In [528]:
from pprint import pprint
import copy
from dictdiffer import diff

group = {}
transactions = {}

def normalizeGroup(dictionary: dict):
    for p1n, p1d in dictionary.items():
        p1d = p1d["balances"]
        for p2n, p2d in dictionary.items():
            p2d = p2d["balances"]
            if p1n != p2n:
                if p1n not in p2d["owes"]:
                    p2d["owes"][p1n] = 0
                if p2n not in p1d["owes"]:
                    p1d["owes"][p2n] = 0
                if p1n not in p2d["owed_by"]:
                    p2d["owed_by"][p1n] = 0
                if p2n not in p1d["owed_by"]:
                    p1d["owed_by"][p2n] = 0

    return dictionary


def addPerson(name):
    group[name] = {}
    group[name]["balances"] = {
        "owes": {},
        "owed_by": {}
    }
    # add total paid by and total owed to
    group[name]["total_owed"] = 0
    group[name]["total_owing"] = 0
    group[name]["balance"] = 0
    normalizeGroup(group)


def createGroup():
    group.clear()
    transactions.clear()
    addPerson("person1")
    addPerson("person2")
    addPerson("person3")

createGroup()
pprint(group)

{'person1': {'balance': 0,
             'balances': {'owed_by': {'person2': 0, 'person3': 0},
                          'owes': {'person2': 0, 'person3': 0}},
             'total_owed': 0,
             'total_owing': 0},
 'person2': {'balance': 0,
             'balances': {'owed_by': {'person1': 0, 'person3': 0},
                          'owes': {'person1': 0, 'person3': 0}},
             'total_owed': 0,
             'total_owing': 0},
 'person3': {'balance': 0,
             'balances': {'owed_by': {'person1': 0, 'person2': 0},
                          'owes': {'person1': 0, 'person2': 0}},
             'total_owed': 0,
             'total_owing': 0}}


In [529]:
def add_transaction(who_got_how_much, who_paid_how_much):
    # the transaction total is the sum of all the values in the who_got_how_much
    transaction_total = sum(who_got_how_much.values())
    pay_total = sum(who_paid_how_much.values())

    # the transaction total must equal the sum of the values in the who_paid_how_much
    if pay_total != transaction_total:
        raise Exception(f"ERROR: pay total {pay_total} does not equal transaction total {transaction_total}")

    transaction = {"who_got_how_much": who_got_how_much,
                   "who_paid_how_much": who_paid_how_much,
                   "transaction_total": transaction_total,
                   "pay_total": pay_total,
                   "prior_state": {}}

    # copy the group's previous state from the previous transaction
    if len(transactions) > 0:
        transaction["prior_state"] = transactions[-1]["prior_state"].copy()
    else:
        for person in group:
            transaction["prior_state"][person] = copy.deepcopy(group[person])

    g1 = copy.deepcopy(transaction["prior_state"])
    # print("=" * 80)
    # print("# Prior State:")
    # pprint(g1)
    prior_state = transaction["prior_state"]
    for p1, owing in who_got_how_much.items():
        for p2, amt_paid in who_paid_how_much.items():

            if p1 != p2:
                value = amt_paid / transaction_total * owing

                transaction["prior_state"][p1]["balances"]["owes"][p2] += value
                transaction["prior_state"][p2]["balances"]["owed_by"][p1] += value

    # Calculate the total owing, total owed, and balance for each person
    for p, v in transaction["prior_state"].items():
        transaction["prior_state"][p]["total_owing"] = sum(v["balances"]["owes"].values())
        transaction["prior_state"][p]["total_owed"] = sum(v["balances"]["owed_by"].values())
        transaction["prior_state"][p]["balance"] = transaction["prior_state"][p]["total_owed"] - transaction["prior_state"][p]["total_owing"]

    g2 = copy.deepcopy(transaction["prior_state"])

    print("-" * 80)
    print("# New State (Only print what's changed):")
    # pprint(g2)
    # Only print the fields in the new state that have changed from the group's prior state
    for person in group:
        diff_dict = list(diff(g1[person], g2[person]))
        if len(diff_dict) > 0:
            print(f"{person}:")
            for d in diff_dict:
                print(f"\t => {d}")

    # Update the group's balances with the new state
    group.update(transaction["prior_state"])
    return transaction

In [530]:
t1 = add_transaction(who_got_how_much={
    'person1': 20,
}, who_paid_how_much={
    'person1': 20,
})

--------------------------------------------------------------------------------
# New State (Only print what's changed):


In [531]:
t2 = add_transaction(who_got_how_much={
    'person1': 20,
}, who_paid_how_much={
    'person2': 20,
})

--------------------------------------------------------------------------------
# New State (Only print what's changed):
person1:
	 => ('change', 'balances.owes.person2', (0, 20.0))
	 => ('change', 'total_owing', (0, 20.0))
	 => ('change', 'balance', (0, -20.0))
person2:
	 => ('change', 'balances.owed_by.person1', (0, 20.0))
	 => ('change', 'total_owed', (0, 20.0))
	 => ('change', 'balance', (0, 20.0))


In [532]:
# TODO: Look at this case to basically clear out the owed by and owes fields
# but the catch is that it needs to be generalized enough to handle if the values are not exactly the same
# (aka in those cases perhaps not clear out but subtract)
t2 = add_transaction(who_got_how_much={
    'person2': 20,
}, who_paid_how_much={
    'person1': 20,
})

--------------------------------------------------------------------------------
# New State (Only print what's changed):
person1:
	 => ('change', 'balances.owed_by.person2', (0, 20.0))
	 => ('change', 'total_owed', (0, 20.0))
	 => ('change', 'balance', (-20.0, 0.0))
person2:
	 => ('change', 'balances.owes.person1', (0, 20.0))
	 => ('change', 'total_owing', (0, 20.0))
	 => ('change', 'balance', (20.0, 0.0))


In [533]:
pprint(group)

{'person1': {'balance': 0.0,
             'balances': {'owed_by': {'person2': 20.0, 'person3': 0},
                          'owes': {'person2': 20.0, 'person3': 0}},
             'total_owed': 20.0,
             'total_owing': 20.0},
 'person2': {'balance': 0.0,
             'balances': {'owed_by': {'person1': 20.0, 'person3': 0},
                          'owes': {'person1': 20.0, 'person3': 0}},
             'total_owed': 20.0,
             'total_owing': 20.0},
 'person3': {'balance': 0,
             'balances': {'owed_by': {'person1': 0, 'person2': 0},
                          'owes': {'person1': 0, 'person2': 0}},
             'total_owed': 0,
             'total_owing': 0}}


In [534]:
t3 = add_transaction(who_got_how_much={
    'person1': 20,
}, who_paid_how_much={
    'person2': 10,
    'person3': 10,
})

--------------------------------------------------------------------------------
# New State (Only print what's changed):
person1:
	 => ('change', 'balances.owes.person2', (20.0, 30.0))
	 => ('change', 'balances.owes.person3', (0, 10.0))
	 => ('change', 'total_owing', (20.0, 40.0))
	 => ('change', 'balance', (0.0, -20.0))
person2:
	 => ('change', 'balances.owed_by.person1', (20.0, 30.0))
	 => ('change', 'total_owed', (20.0, 30.0))
	 => ('change', 'balance', (0.0, 10.0))
person3:
	 => ('change', 'balances.owed_by.person1', (0, 10.0))
	 => ('change', 'total_owed', (0, 10.0))
	 => ('change', 'balance', (0, 10.0))


In [535]:
t4 = add_transaction(who_got_how_much={
    'person1': 10,
    'person3': 10,
}, who_paid_how_much={
    'person1': 20,
})

--------------------------------------------------------------------------------
# New State (Only print what's changed):
person1:
	 => ('change', 'balances.owed_by.person3', (0, 10.0))
	 => ('change', 'total_owed', (20.0, 30.0))
	 => ('change', 'balance', (-20.0, -10.0))
person3:
	 => ('change', 'balances.owes.person1', (0, 10.0))
	 => ('change', 'total_owing', (0, 10.0))
	 => ('change', 'balance', (10.0, 0.0))


In [536]:
t5 = add_transaction(who_got_how_much={
    'person2': 30,
}, who_paid_how_much={
    'person2': 30,
})

--------------------------------------------------------------------------------
# New State (Only print what's changed):


In [537]:
t5 = add_transaction(who_got_how_much={
    'person2': 30,
    'person3': 30,
}, who_paid_how_much={
    'person2': 30,
    'person3': 30,
})

--------------------------------------------------------------------------------
# New State (Only print what's changed):
person2:
	 => ('change', 'balances.owes.person3', (0, 15.0))
	 => ('change', 'balances.owed_by.person3', (0, 15.0))
	 => ('change', 'total_owed', (30.0, 45.0))
	 => ('change', 'total_owing', (20.0, 35.0))
person3:
	 => ('change', 'balances.owes.person2', (0, 15.0))
	 => ('change', 'balances.owed_by.person2', (0, 15.0))
	 => ('change', 'total_owed', (10.0, 25.0))
	 => ('change', 'total_owing', (10.0, 25.0))
